In [171]:
import pandas as pd
import numpy as np
import os

import joblib

from keras import utils
from sklearn.model_selection import train_test_split, GridSearchCV
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D, MaxPooling2D, Conv2D, LSTM, GRU, Bidirectional
from keras import regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
import keras

In [172]:
df = pd.read_csv('NonPromoterSequence.txt', sep = '>', )
df.dropna(subset=['Unnamed: 0'], how='all', inplace=True)
df.reset_index(inplace = True)
df.drop(['EP 1 (+) mt:CoI_1; range -400 to -100.', 'index'], axis = 1, inplace=True)
df.rename(columns={'Unnamed: 0': "sequence"}, inplace = True)
df['label'] = 0
display(df)
display(df.shape)

,sequence,label
0,TAATTACATTATTTTTTTATTTACGAATTTGTTATTCCGCTTTTAT...,0
1,ATTTTTACAAGAACAAGACATTTAACTTTAACTTTATCTTTAGCTT...,0
2,AGAGATAGGTGGGTCTGTAACACTCGAATCAAAAACAATATTAAGA...,0
3,TATGTATATAGAGATAGGCGTTGCCAATAACTTTTGCGTTTTTTGC...,0
4,AGAAATAATAGCTAGAGCAAAAAACAGCTTAGAACGGCTGATGCTC...,0
...,...,...
11295,TGGTAAAAAATTGTACACCTAACTAGTGCCTTCATGTATACCACCA...,0
11296,AGTGCAACTGGAGCCGTGCCGTGACCCACAGAGATCGCCCACTCGA...,0
11297,GCATGGATTTCATATTATCTTAATCGACTTGCTTTTATAAAATAGG...,0
11298,GTGACCAGGTTTTGCTCTAATGCGAAGTACGGATTGGGTAGAGATA...,0


(11300, 2)

In [173]:
df2 = pd.read_csv('PromoterSequence.txt', sep = '>', )
df2.dropna(subset=['Unnamed: 0'], how='all', inplace=True)
df2.reset_index(inplace = True)
df2.drop(['EP 1 (+) mt:CoI_1; range -100 to 200.', 'index'], axis = 1, inplace=True)
df2.rename(columns={'Unnamed: 0': "sequence"}, inplace = True)
df2['label'] = 1

display(df2)
display(df2.shape)

,sequence,label
0,TTAATTTGTCCTTATTTGATTAAGAAGAATAAATCTTATATATAGA...,1
1,ATAGCTCAAATTGCTTTATTAGTATTAGAATCAGCTGTAGCTATAA...,1
2,AAGCTTCCCTTTAATGTGCTCCTTGTGAATACAGCATTACAATGCC...,1
3,TATGTAGAATCTGTACAAGTATCTGTGTTTGGACAATGGCATGTGT...,1
4,ACATATTACTGCATACAGGTCTCAAATTATAAAATGACACTCGTGG...,1
...,...,...
11295,CGACAAAGTTTGATCCATGTGCATTCTTGGCGCCTTATCGATAGCT...,1
11296,CATATCTACATCTCGCTTGCTCCTTCCCTTTCGCTGCGTGTGTGTG...,1
11297,ATACCGCGGAAGCGCAAAAGTACCAGAATTTCCCTGGTATCGCGCT...,1
11298,ATTATTCCGAATTCTTTTATCAGATTTAAATATGGGAAACACTTTA...,1


(11300, 2)

In [174]:
df = pd.concat([df, df2], axis = 0 )
df.shape

(22600, 2)

In [175]:
for seq in df['sequence']:
    if 'N' in seq:
        display(df.loc[df['sequence'] == seq])

,sequence,label
1822,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNGAATTC...,0


In [176]:
df.drop([1822], inplace = True)

In [177]:
for seq in df['sequence']:
    if 'N' in seq:
        display(df.loc[df['sequence'] == seq])

In [178]:
sequence = list(df.loc[:, 'sequence'])
encoded_list = []

In [179]:
def encode_seq(s):
    Encode = {'A':[1,0,0,0],'T':[0,1,0,0],'C':[0,0,1,0],'G':[0,0,0,1]}
    return [Encode[x] for x in s]

for i in sequence:
    x = encode_seq(i)
    encoded_list.append(x)

X = np.array(encoded_list)
X.shape

(22598, 301, 4)

X = X.reshape(X.shape[0],301, 4, 1)
X.shape

In [180]:
y = df['label']
y.shape

(22598,)

In [181]:
X.shape

(22598, 301, 4)

In [182]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, stratify = y)

In [183]:
y_train = utils.to_categorical(y_train)
y_test = utils.to_categorical(y_test)

In [184]:
params = {
    'first_node': [128, 64],
    'second_node': [32, 64],
    'alpha': [0.001, 0.01],
    'first_filter': [9, 16, 32], 
    'dropout': [0.1, 0.2, 0.5]
    
    
}

model = gs.best_estimator_

gs.best_params_

In [197]:
gru_model = Sequential()

gru_model.add(Conv1D(filters = 27, kernel_size = (4), activation = 'relu', input_shape = (301, 4)))
gru_model.add(MaxPooling1D(pool_size= (3)))
gru_model.add(Dropout(0.2))

gru_model.add(Conv1D(filters = 14, kernel_size = (2), activation = 'relu', padding = 'same'))
#cnn_model.add(MaxPooling1D(pool_size= (1)))
#cnn_model.add(Dropout(0.2))



gru_model.add(Bidirectional(GRU(128, activation = 'relu')))
gru_model.add(Dropout(0.2))
gru_model.add(Dense(128, activation = 'relu'))
gru_model.add(Dense(64, activation = 'relu'))
gru_model.add(Dense(64, activation = 'relu'))
gru_model.add(Dense(16, activation = 'relu', kernel_regularizer = regularizers.l2(0.01)))
gru_model.add(Dense(2, activation = 'sigmoid'))

gru_model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

early_stop = keras.callbacks.EarlyStopping(monitor = 'val_accuracy', min_delta = 0.0005, patience=8, 
                                           restore_best_weights=True )
history = gru_model.fit(X_train, y_train, batch_size = 128, validation_data=(X_test, y_test), 
                        epochs=115)

Epoch 1/115
133/133 [==============================] - 25s 176ms/step - loss: 0.8568 - accuracy: 0.5949 - val_loss: 0.6412 - val_accuracy: 0.7002
Epoch 2/115
133/133 [==============================] - 24s 182ms/step - loss: 0.5991 - accuracy: 0.7225 - val_loss: 0.5443 - val_accuracy: 0.7384
Epoch 3/115
133/133 [==============================] - 25s 185ms/step - loss: 0.5427 - accuracy: 0.7420 - val_loss: 0.5373 - val_accuracy: 0.7354
Epoch 4/115
133/133 [==============================] - 25s 185ms/step - loss: 0.5223 - accuracy: 0.7488 - val_loss: 0.5212 - val_accuracy: 0.7441
Epoch 5/115
133/133 [==============================] - 25s 187ms/step - loss: 0.5113 - accuracy: 0.7539 - val_loss: 0.5068 - val_accuracy: 0.7536
Epoch 6/115
133/133 [==============================] - 25s 189ms/step - loss: 0.5115 - accuracy: 0.7522 - val_loss: 0.5016 - val_accuracy: 0.7558
Epoch 7/115
133/133 [==============================] - 25s 188ms/step - loss: 0.5089 - accuracy: 0.7554 - val_loss: 0.5048 -

Epoch 57/115
133/133 [==============================] - 25s 191ms/step - loss: 0.2828 - accuracy: 0.8813 - val_loss: 0.3203 - val_accuracy: 0.8685
Epoch 58/115
133/133 [==============================] - 25s 189ms/step - loss: 0.2792 - accuracy: 0.8842 - val_loss: 0.3136 - val_accuracy: 0.8688
Epoch 59/115
133/133 [==============================] - 25s 189ms/step - loss: 0.2760 - accuracy: 0.8803 - val_loss: 0.3137 - val_accuracy: 0.8713
Epoch 60/115
133/133 [==============================] - 25s 188ms/step - loss: 0.2821 - accuracy: 0.8836 - val_loss: 0.3497 - val_accuracy: 0.8506
Epoch 61/115
133/133 [==============================] - 25s 189ms/step - loss: 0.2717 - accuracy: 0.8860 - val_loss: 0.3128 - val_accuracy: 0.8688
Epoch 62/115
133/133 [==============================] - 25s 190ms/step - loss: 0.2724 - accuracy: 0.8836 - val_loss: 0.3331 - val_accuracy: 0.8731
Epoch 63/115
133/133 [==============================] - 25s 189ms/step - loss: 0.2797 - accuracy: 0.8811 - val_loss: 0

133/133 [==============================] - 25s 187ms/step - loss: 0.1827 - accuracy: 0.9235 - val_loss: 0.3261 - val_accuracy: 0.8809
Epoch 113/115
133/133 [==============================] - 25s 187ms/step - loss: 0.1829 - accuracy: 0.9274 - val_loss: 0.3311 - val_accuracy: 0.8699
Epoch 114/115
133/133 [==============================] - 25s 188ms/step - loss: 0.1875 - accuracy: 0.9245 - val_loss: 0.3221 - val_accuracy: 0.8830
Epoch 115/115
133/133 [==============================] - 25s 188ms/step - loss: 0.1754 - accuracy: 0.9307 - val_loss: 0.3335 - val_accuracy: 0.8835


In [ ]:
pred = gru_model.predict

In [201]:
df = pd.read_csv('test.csv', sep = '\n', )
df

,Sequence
0,AATCTCTGTTGTTGTTATGCAAAAAAGGAATAATGTAATGTATTAT...
1,TTCTCTAAGAAAGTCGATGCTAAGCGGATGCTAAACGCAGGGCCGG...
2,ACATTTTACGGTCTTGCGTGTGTATTTATTGTTCGTACAAGTCAAA...
3,CACTGAAAAAAAAAGAAAGGCTTATTTACTATTAACAACTTTAATA...
4,GGGAGTGGCAACATGGGCTCACAAGTCTAGATCGACTCGGCTATTG...
...,...
173,TAAATTGCATGTAAAATCATTAATAACGATTATATTCAGTTTGGCC...
174,GGATGTGCTTAGTGTAATTTGCTTATAAAAACTACTTTATATTTTC...
175,ATTCAGAAATTACATGTTTCTGAAAACTCTTTCCGATCAAAAACAA...
176,ACGGTTTTAAGTGCCCAAACTTAGGGTGTAGCGCCCTTTTCAGTTC...


In [202]:
sequence = list(df.loc[:, 'Sequence'])
encoded_list = []

In [204]:
def encode_seq(s):
    Encode = {'A':[1,0,0,0],'T':[0,1,0,0],'C':[0,0,1,0],'G':[0,0,0,1]}
    return [Encode[x] for x in s]

for i in sequence:
    x = encode_seq(i)
    encoded_list.append(x)

X_test_test = np.array(encoded_list)
X_test_test.shape

(178, 301, 4)

In [207]:
preds = gru_model.predict_classes(X_test_test)

/Users/mathea/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [208]:
preds

array([0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1,
       1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0])

In [210]:
len(preds)

178

In [209]:
np.savetxt("test_predictions.csv", preds, delimiter=",")

## Initial Gridsearch on CNN model as backup.

The models is saved in this file. 


Best Parameters are
 - L2 alpha of 0.01, 
 - Dropout of 0.02
 - First filter: 32
 - First node: 64
 - Second node: 64

In [ ]:
def model_func(first_node, second_node, alpha, first_filter, dropout):

    model = Sequential()
    model.add(Conv2D(filters = first_filter,       
                    kernel_size = (3,3),    
                    activation = 'relu',    
                  input_shape = (301, 4,1)))
    model.add(MaxPooling2D(pool_size= (2,2)))
    model.add(Dropout(dropout))

    model.add(Conv2D(filters = 15,  
                     kernel_size = (2,2),
                    activation = 'relu',
                    padding = 'same'))
    model.add(MaxPooling2D(pool_size= (1,1)))
    model.add(Dropout(dropout))


    model.add(Flatten())
    model.add(Dense(first_node, activation = 'relu', kernel_regularizer = regularizers.l2(alpha)))
    model.add(Dense(second_node, activation = 'relu'))
    model.add(Dense(32, activation = 'relu'))
    model.add(Dense(16, activation = 'relu'))
    model.add(Dense(2, activation = 'sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    
    return model

In [16]:
early_stop = keras.callbacks.EarlyStopping(monitor = 'loss', min_delta = 0, patience=5, 
                                           restore_best_weights=True )


nn = KerasClassifier(build_fn = model_func, batch_size = 512, verbose = 2, epochs = 200)

gs = GridSearchCV(nn, param_grid = params, cv = 5 )

gs.fit(X_train, y_train, callbacks = [early_stop])

Epoch 1/200
27/27 - 2s - loss: 0.7893 - accuracy: 0.6577
Epoch 2/200
27/27 - 1s - loss: 0.5753 - accuracy: 0.7497
Epoch 3/200
27/27 - 1s - loss: 0.5266 - accuracy: 0.7688
Epoch 4/200
27/27 - 1s - loss: 0.5090 - accuracy: 0.7775
Epoch 5/200
27/27 - 1s - loss: 0.4856 - accuracy: 0.7913
Epoch 6/200
27/27 - 1s - loss: 0.4880 - accuracy: 0.7905
Epoch 7/200
27/27 - 1s - loss: 0.4640 - accuracy: 0.8022
Epoch 8/200
27/27 - 1s - loss: 0.4456 - accuracy: 0.8194
Epoch 9/200
27/27 - 1s - loss: 0.4419 - accuracy: 0.8161
Epoch 10/200
27/27 - 1s - loss: 0.4478 - accuracy: 0.8140
Epoch 11/200
27/27 - 2s - loss: 0.4357 - accuracy: 0.8195
Epoch 12/200
27/27 - 1s - loss: 0.4183 - accuracy: 0.8286
Epoch 13/200
27/27 - 1s - loss: 0.4196 - accuracy: 0.8284
Epoch 14/200
27/27 - 1s - loss: 0.4129 - accuracy: 0.8315
Epoch 15/200
27/27 - 1s - loss: 0.4000 - accuracy: 0.8416
Epoch 16/200
27/27 - 1s - loss: 0.3927 - accuracy: 0.8439
Epoch 17/200
27/27 - 1s - loss: 0.3899 - accuracy: 0.8443
Epoch 18/200
27/27 - 2s

Epoch 53/200
27/27 - 1s - loss: 0.2339 - accuracy: 0.9169
Epoch 54/200
27/27 - 1s - loss: 0.2423 - accuracy: 0.9147
Epoch 55/200
27/27 - 1s - loss: 0.2279 - accuracy: 0.9182
Epoch 56/200
27/27 - 1s - loss: 0.2144 - accuracy: 0.9306
Epoch 57/200
27/27 - 1s - loss: 0.2107 - accuracy: 0.9285
Epoch 58/200
27/27 - 1s - loss: 0.2130 - accuracy: 0.9275
Epoch 59/200
27/27 - 1s - loss: 0.2049 - accuracy: 0.9327
Epoch 60/200
27/27 - 1s - loss: 0.2066 - accuracy: 0.9306
Epoch 61/200
27/27 - 1s - loss: 0.1957 - accuracy: 0.9372
Epoch 62/200
27/27 - 1s - loss: 0.2015 - accuracy: 0.9339
Epoch 63/200
27/27 - 1s - loss: 0.1895 - accuracy: 0.9406
Epoch 64/200
27/27 - 1s - loss: 0.1818 - accuracy: 0.9455
Epoch 65/200
27/27 - 1s - loss: 0.1791 - accuracy: 0.9462
Epoch 66/200
27/27 - 1s - loss: 0.1694 - accuracy: 0.9506
Epoch 67/200
27/27 - 1s - loss: 0.1628 - accuracy: 0.9539
Epoch 68/200
27/27 - 1s - loss: 0.1695 - accuracy: 0.9496
Epoch 69/200
27/27 - 1s - loss: 0.1555 - accuracy: 0.9562
Epoch 70/200
2

Epoch 8/200
27/27 - 1s - loss: 0.4351 - accuracy: 0.8255
Epoch 9/200
27/27 - 1s - loss: 0.4360 - accuracy: 0.8213
Epoch 10/200
27/27 - 1s - loss: 0.4304 - accuracy: 0.8220
Epoch 11/200
27/27 - 1s - loss: 0.4132 - accuracy: 0.8339
Epoch 12/200
27/27 - 1s - loss: 0.4073 - accuracy: 0.8360
Epoch 13/200
27/27 - 1s - loss: 0.4033 - accuracy: 0.8358
Epoch 14/200
27/27 - 1s - loss: 0.4044 - accuracy: 0.8372
Epoch 15/200
27/27 - 1s - loss: 0.3855 - accuracy: 0.8442
Epoch 16/200
27/27 - 1s - loss: 0.3709 - accuracy: 0.8535
Epoch 17/200
27/27 - 1s - loss: 0.3694 - accuracy: 0.8519
Epoch 18/200
27/27 - 1s - loss: 0.3654 - accuracy: 0.8581
Epoch 19/200
27/27 - 1s - loss: 0.3657 - accuracy: 0.8526
Epoch 20/200
27/27 - 1s - loss: 0.3615 - accuracy: 0.8537
Epoch 21/200
27/27 - 1s - loss: 0.3602 - accuracy: 0.8569
Epoch 22/200
27/27 - 1s - loss: 0.3471 - accuracy: 0.8649
Epoch 23/200
27/27 - 1s - loss: 0.3348 - accuracy: 0.8717
Epoch 24/200
27/27 - 1s - loss: 0.3347 - accuracy: 0.8677
Epoch 25/200
27/

Epoch 68/200
27/27 - 1s - loss: 0.1739 - accuracy: 0.9471
Epoch 69/200
27/27 - 1s - loss: 0.1635 - accuracy: 0.9534
Epoch 70/200
27/27 - 1s - loss: 0.1633 - accuracy: 0.9532
Epoch 71/200
27/27 - 1s - loss: 0.1664 - accuracy: 0.9524
Epoch 72/200
27/27 - 1s - loss: 0.1655 - accuracy: 0.9518
Epoch 73/200
27/27 - 1s - loss: 0.1526 - accuracy: 0.9584
Epoch 74/200
27/27 - 1s - loss: 0.1493 - accuracy: 0.9602
Epoch 75/200
27/27 - 1s - loss: 0.1482 - accuracy: 0.9599
Epoch 76/200
27/27 - 1s - loss: 0.1541 - accuracy: 0.9578
Epoch 77/200
27/27 - 1s - loss: 0.1398 - accuracy: 0.9653
Epoch 78/200
27/27 - 1s - loss: 0.1414 - accuracy: 0.9639
Epoch 79/200
27/27 - 1s - loss: 0.1483 - accuracy: 0.9603
Epoch 80/200
27/27 - 1s - loss: 0.1368 - accuracy: 0.9644
Epoch 81/200
27/27 - 1s - loss: 0.1420 - accuracy: 0.9656
Epoch 82/200
27/27 - 1s - loss: 0.1381 - accuracy: 0.9667
Epoch 83/200
27/27 - 1s - loss: 0.1452 - accuracy: 0.9625
Epoch 84/200
27/27 - 1s - loss: 0.1322 - accuracy: 0.9681
Epoch 85/200
2

Epoch 91/200
27/27 - 1s - loss: 0.1339 - accuracy: 0.9673
Epoch 92/200
27/27 - 1s - loss: 0.1388 - accuracy: 0.9642
Epoch 93/200
27/27 - 1s - loss: 0.1345 - accuracy: 0.9636
Epoch 94/200
27/27 - 1s - loss: 0.1369 - accuracy: 0.9644
7/7 - 0s - loss: 0.5505 - accuracy: 0.8404
Epoch 1/200
27/27 - 2s - loss: 0.7874 - accuracy: 0.6469
Epoch 2/200
27/27 - 1s - loss: 0.5723 - accuracy: 0.7624
Epoch 3/200
27/27 - 1s - loss: 0.5235 - accuracy: 0.7818
Epoch 4/200
27/27 - 1s - loss: 0.4903 - accuracy: 0.7959
Epoch 5/200
27/27 - 1s - loss: 0.4632 - accuracy: 0.8118
Epoch 6/200
27/27 - 1s - loss: 0.4488 - accuracy: 0.8170
Epoch 7/200
27/27 - 1s - loss: 0.4356 - accuracy: 0.8251
Epoch 8/200
27/27 - 1s - loss: 0.4228 - accuracy: 0.8309
Epoch 9/200
27/27 - 1s - loss: 0.4078 - accuracy: 0.8394
Epoch 10/200
27/27 - 1s - loss: 0.4122 - accuracy: 0.8352
Epoch 11/200
27/27 - 1s - loss: 0.4126 - accuracy: 0.8321
Epoch 12/200
27/27 - 1s - loss: 0.4001 - accuracy: 0.8416
Epoch 13/200
27/27 - 1s - loss: 0.3882

...

Epoch 38/200
27/27 - 3s - loss: 0.4130 - accuracy: 0.8305
Epoch 39/200
27/27 - 3s - loss: 0.4137 - accuracy: 0.8317
Epoch 40/200
27/27 - 3s - loss: 0.4155 - accuracy: 0.8286
Epoch 41/200
27/27 - 3s - loss: 0.4092 - accuracy: 0.8306
Epoch 42/200
27/27 - 3s - loss: 0.4009 - accuracy: 0.8364
Epoch 43/200
27/27 - 3s - loss: 0.4057 - accuracy: 0.8350
Epoch 44/200
27/27 - 3s - loss: 0.4042 - accuracy: 0.8359
Epoch 45/200
27/27 - 3s - loss: 0.3959 - accuracy: 0.8412
Epoch 46/200
27/27 - 3s - loss: 0.3953 - accuracy: 0.8399
Epoch 47/200
27/27 - 3s - loss: 0.4031 - accuracy: 0.8355
Epoch 48/200
27/27 - 3s - loss: 0.3961 - accuracy: 0.8448
Epoch 49/200
27/27 - 3s - loss: 0.4008 - accuracy: 0.8384
Epoch 50/200
27/27 - 3s - loss: 0.3954 - accuracy: 0.8434
Epoch 51/200
27/27 - 3s - loss: 0.3917 - accuracy: 0.8431
Epoch 52/200
27/27 - 3s - loss: 0.3949 - accuracy: 0.8429
Epoch 53/200
27/27 - 3s - loss: 0.3992 - accuracy: 0.8407
Epoch 54/200
27/27 - 3s - loss: 0.3999 - accuracy: 0.8406
Epoch 55/200
2

Epoch 9/200
27/27 - 3s - loss: 0.5080 - accuracy: 0.7714
Epoch 10/200
27/27 - 3s - loss: 0.4941 - accuracy: 0.7779
Epoch 11/200
27/27 - 3s - loss: 0.5082 - accuracy: 0.7714
Epoch 12/200
27/27 - 3s - loss: 0.4882 - accuracy: 0.7863
Epoch 13/200
27/27 - 3s - loss: 0.4712 - accuracy: 0.7953
Epoch 14/200
27/27 - 3s - loss: 0.4689 - accuracy: 0.7958
Epoch 15/200
27/27 - 3s - loss: 0.4672 - accuracy: 0.7960
Epoch 16/200
27/27 - 3s - loss: 0.4735 - accuracy: 0.7944
Epoch 17/200
27/27 - 3s - loss: 0.4556 - accuracy: 0.8009
Epoch 18/200
27/27 - 3s - loss: 0.4576 - accuracy: 0.8027
Epoch 19/200
27/27 - 3s - loss: 0.4462 - accuracy: 0.8107
Epoch 20/200
27/27 - 3s - loss: 0.4502 - accuracy: 0.8053
Epoch 21/200
27/27 - 3s - loss: 0.4473 - accuracy: 0.8073
Epoch 22/200
27/27 - 3s - loss: 0.4452 - accuracy: 0.8091
Epoch 23/200
27/27 - 3s - loss: 0.4366 - accuracy: 0.8152
Epoch 24/200
27/27 - 3s - loss: 0.4384 - accuracy: 0.8160
Epoch 25/200
27/27 - 3s - loss: 0.4388 - accuracy: 0.8123
Epoch 26/200
27

Epoch 6/200
27/27 - 3s - loss: 0.5270 - accuracy: 0.7577
Epoch 7/200
27/27 - 3s - loss: 0.5275 - accuracy: 0.7583
Epoch 8/200
27/27 - 3s - loss: 0.5107 - accuracy: 0.7687
Epoch 9/200
27/27 - 3s - loss: 0.4970 - accuracy: 0.7782
Epoch 10/200
27/27 - 3s - loss: 0.4892 - accuracy: 0.7822
Epoch 11/200
27/27 - 3s - loss: 0.4843 - accuracy: 0.7853
Epoch 12/200
27/27 - 3s - loss: 0.4842 - accuracy: 0.7891
Epoch 13/200
27/27 - 3s - loss: 0.4731 - accuracy: 0.7903
Epoch 14/200
27/27 - 3s - loss: 0.4669 - accuracy: 0.7977
Epoch 15/200
27/27 - 3s - loss: 0.4704 - accuracy: 0.7919
Epoch 16/200
27/27 - 3s - loss: 0.4538 - accuracy: 0.8039
Epoch 17/200
27/27 - 3s - loss: 0.4585 - accuracy: 0.7983
Epoch 18/200
27/27 - 3s - loss: 0.4455 - accuracy: 0.8085
Epoch 19/200
27/27 - 3s - loss: 0.4559 - accuracy: 0.7990
Epoch 20/200
27/27 - 3s - loss: 0.4376 - accuracy: 0.8155
Epoch 21/200
27/27 - 3s - loss: 0.4449 - accuracy: 0.8076
Epoch 22/200
27/27 - 3s - loss: 0.4341 - accuracy: 0.8163
Epoch 23/200
27/27

Epoch 101/200
34/34 - 3s - loss: 0.2307 - accuracy: 0.9396
Epoch 102/200
34/34 - 3s - loss: 0.2233 - accuracy: 0.9427
Epoch 103/200
34/34 - 3s - loss: 0.2153 - accuracy: 0.9473
Epoch 104/200
34/34 - 3s - loss: 0.2321 - accuracy: 0.9409
Epoch 105/200
34/34 - 3s - loss: 0.2219 - accuracy: 0.9443
Epoch 106/200
34/34 - 3s - loss: 0.2182 - accuracy: 0.9452
Epoch 107/200
34/34 - 3s - loss: 0.2229 - accuracy: 0.9432
Epoch 108/200
34/34 - 3s - loss: 0.2218 - accuracy: 0.9445


GridSearchCV(cv=5,
             estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f9765becb20>,
             param_grid={'alpha': [0.001, 0.01], 'dropout': [0.1, 0.2, 0.5],
                         'first_filter': [9, 16, 32], 'first_node': [128, 64],
                         'second_node': [32, 64]})

In [21]:
cnn_model_saved = cnn_model.save('CNN_best_model.h5')